# Variable Inspector Widget

## A short example implementation

This notebook demonstrates how one can use the widgets already built in to IPython to create a working variable inspector much like the ones seen in popular commercial scientific computing environments.

In [1]:
import ipywidgets as widgets # Loads the Widget framework.
from IPython.core.magics.namespace import NamespaceMagics # Used to query namespace.

# For this example, hide these names, just to avoid polluting the namespace further
get_ipython().user_ns_hidden['widgets'] = widgets
get_ipython().user_ns_hidden['NamespaceMagics'] = NamespaceMagics

In [2]:
class VariableInspectorWindow(object):
    instance = None
    
    def __init__(self, ipython):
        """Public constructor."""
        if VariableInspectorWindow.instance is not None:
            raise Exception("""Only one instance of the Variable Inspector can exist at a 
                time.  Call close() on the active instance before creating a new instance.
                If you have lost the handle to the active instance, you can re-obtain it
                via `VariableInspectorWindow.instance`.""")
        
        VariableInspectorWindow.instance = self
        self.closed = False
        self.namespace = NamespaceMagics()
        self.namespace.shell = ipython.kernel.shell
        
        self._box = widgets.Box()
        self._box._dom_classes = ['inspector']
        self._box.background_color = '#fff'
        self._box.border_color = '#ccc'
        self._box.border_width = 1
        self._box.border_radius = 5

        self._modal_body = widgets.VBox()
        self._modal_body.overflow_y = 'scroll'

        self._modal_body_label = widgets.HTML(value = 'Not hooked')
        self._modal_body.children = [self._modal_body_label]

        self._box.children = [
            self._modal_body, 
        ]
        
        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)
        
    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            self._ipython.events.unregister('post_run_cell', self._fill)
            self._box.close()
            self.closed = True
            VariableInspectorWindow.instance = None

    def _fill(self):
        """Fill self with variable information."""
        values = self.namespace.who_ls()
        self._modal_body_label.value = '<table class="table table-bordered table-striped"><tr><th>Name</th><th>Type</th><th>Value</th></tr><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(eval(v)).__name__, str(eval(v))) for v in values]) + \
            '</td></tr></table>'

    def _ipython_display_(self):
        """Called when display() or pyout is used to display the Variable 
        Inspector."""
        self._box._ipython_display_()


In [3]:
inspector = VariableInspectorWindow(get_ipython())
inspector

Pop the inspector out of the widget area using Javascript.  To close the inspector, click the close button on the widget area that it was spawned from.

In [4]:
%%javascript
$('div.inspector')
    .detach()
    .prependTo($('body'))
    .css({
        'z-index': 999, 
        position: 'fixed',
        'box-shadow': '5px 5px 12px -3px black',
        opacity: 0.9
    })
    .draggable();

<IPython.core.display.Javascript object>

# Test

In [5]:
a = 5

In [6]:
b = 3.0

In [7]:
c = a * b

In [8]:
d = "String"

In [9]:
del b

In [10]:
inspector.close()